<!-- TABS -->
# Text Vector Search

You'll find this example as well as the saved template in the main repository of `superduper`.
See [here](https://github.com/superduper-io/superduper/tree/main/templates/text_vector_search).

If you'd like to modify the template, or practice building it yourself, then you can rerun the `build.ipynb` notebook
in the template directory

<!-- TABS -->
## Connect to superduper

In [1]:
from superduper import superduper

db = superduper('mongomock://test_db')

2024-Aug-23 14:38:44.94| INFO     | Duncans-MacBook-Pro.local| superduper.misc.plugins:13   | Loading plugin: mongodb
2024-Aug-23 14:38:45.01| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:103  | Building Data Layer
2024-Aug-23 14:38:45.01| INFO     | Duncans-MacBook-Pro.local| superduper.base.build:171  | Configuration: 
 +---------------+---------------------+
| Configuration |        Value        |
+---------------+---------------------+
|  Data Backend | mongomock://test_db |
+---------------+---------------------+


<!-- TABS -->
## Get useful sample data

In [2]:
# <tab: Text>
!curl -O https://superduperdb-public-demo.s3.amazonaws.com/text.json
import json

with open('text.json', 'r') as f:
    data = json.load(f)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  720k  100  720k    0     0   277k      0  0:00:02  0:00:02 --:--:--  278k


In [ ]:
# <tab: PDF>
!curl -O https://superduperdb-public-demo.s3.amazonaws.com/pdfs.zip && unzip -o pdfs.zip
import os

data = [f'pdfs/{x}' for x in os.listdir('./pdfs') if x.endswith('.pdf')]

In [3]:
datas = [{'x': d} for d in data]

<!-- TABS -->
## Create datatype

SuperduperDB supports automatic data conversion, so users don’t need to worry about the compatibility of different data formats (`PIL.Image`, `numpy.array`, `pandas.DataFrame`, etc.) with the database.

It also supports custom data conversion methods for transforming data, such as defining the following Datatype.

In [4]:
# <tab: Text>
datatype = 'str'

In [ ]:
# <tab: PDF>
from superduper import DataType

# By creating a datatype and setting its encodable attribute to “file” for saving PDF files, 
# all datatypes encoded as “file” will have their corresponding files uploaded to the artifact store. 
# References will be recorded in the database, and the files will be downloaded locally when needed. 

datatype = DataType('pdf', encodable='file')

<!-- TABS -->
## Setup tables or collections

In [5]:
from superduper.components.table import Table
from superduper import Schema

schema = Schema(identifier="schema", fields={"x": datatype})
table = Table("docs", schema=schema)
select = db['docs'].select()

<!-- TABS -->
## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [6]:
# <tab: Text>
from superduper import model

CHUNK_SIZE = 200

@model(flatten=True, model_update_kwargs={'document_embedded': False})
def chunker(text):
    text = text.split()
    chunks = [' '.join(text[i:i + CHUNK_SIZE]) for i in range(0, len(text), CHUNK_SIZE)]
    return chunks

In [ ]:
# <tab: PDF>
!pip install -q "unstructured[pdf]"
from superduper import model
from unstructured.partition.pdf import partition_pdf

CHUNK_SIZE = 500

@model(flatten=True)
def chunker(pdf_file):
    elements = partition_pdf(pdf_file)
    text = '\n'.join([e.text for e in elements])
    chunks = [text[i:i + CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]
    return chunks

Now we wrap this chunker as a `Listener`, so that it processes incoming data

In [7]:
from superduper import Listener

upstream_listener = Listener(
    model=chunker,
    select=db['docs'].select(),
    key='x',
    uuid="chunk",
    identifier='chunker',
)

## Select outputs of upstream listener

:::note
This is useful if you have performed a first step, such as pre-computing 
features, or chunking your data. You can use this query to 
operate on those outputs.
:::

In [8]:
indexing_key = upstream_listener.outputs
indexing_key

'_outputs__chunker'

<!-- TABS -->
## Build text embedding model

In [9]:
# <tab: OpenAI>
from superduper_openai import OpenAIEmbedding
import os

os.environ['OPENAI_API_KEY'] = 'sk-<secret>'

embedding_model = OpenAIEmbedding(identifier='text-embedding-ada-002')

In [ ]:
# <tab: JinaAI>
import os
from superduper_jina import JinaEmbedding

os.environ["JINA_API_KEY"] = "jina_xxxx"
 
# define the model
embedding_model = JinaEmbedding(identifier='jina-embeddings-v2-base-en')

In [ ]:
# <tab: Sentence-Transformers>
!pip install sentence-transformers
from superduper import vector
import sentence_transformers
from superduper_sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(
    identifier="embedding",
    object=sentence_transformers.SentenceTransformer("BAAI/bge-small-en"),
    datatype=vector(shape=(1024,)),
    postprocess=lambda x: x.tolist(),
    predict_kwargs={"show_progress_bar": True},
)

In [ ]:
print(len(embedding_model.predict("What is superduper")))

## Create vector-index

In [10]:
vector_index_name = 'my-vector-index'

In [11]:
from superduper import VectorIndex, Listener

vector_index = VectorIndex(
    vector_index_name,
    indexing_listener=Listener(
        key=indexing_key,              # the `Document` key `model` should ingest to create embedding
        select=db[indexing_key].select(),                 # a `Select` query telling which data to search over
        model=embedding_model,         # a `_Predictor` how to convert data to embeddings
        identifier=f'{embedding_model.identifier}-listener',
        upstream=[table, upstream_listener],              # this makes sure that the table is already set up when the other components are triggered
    )
)

In [13]:
from superduper import Application

application = Application(
    'text-vector-search', 
    components=[
        table,
        upstream_listener,
        vector_index,
    ]
)

In [14]:
db.apply(application)

2024-Aug-23 14:40:30.87| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf str already exists
2024-Aug-23 14:40:30.87| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf str already exists
2024-Aug-23 14:40:30.87| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf str already exists
2024-Aug-23 14:40:30.88| INFO     | Duncans-MacBook-Pro.local| superduper.components.listener:94   | Requesting listener setup on CDC service
2024-Aug-23 14:40:30.88| INFO     | Duncans-MacBook-Pro.local| superduper.components.listener:104  | Skipping listener setup on CDC service since no URI is set
2024-Aug-23 14:40:30.90| INFO     | Duncans-MacBook-Pro.local| superduper.components.listener:94   | Requesting listener setup on CDC service
2024-Aug-23 14:40:30.90| INFO     | Duncans-MacBook-Pro.local| superduper.components.listener:104  | Skipping listener setup on CDC service since no URI is set
2024-Aug-23 14:40:30.92| INFO     | Dun

Loading vectors into vector-table...: 0it [00:00, ?it/s]

2024-Aug-23 14:40:30.92| INFO     | Duncans-MacBook-Pro.local| superduper.components.vector_index:97   | Loaded 0 vectors into vector index succesfully


([],
 Application(identifier='text-vector-search', uuid='ba67055ba77d40ffa73d5cd3a17118aa', upstream=None, plugins=None, cache=False, components=[Table(identifier='documents', uuid='d05bd908810a43988dbe41c2167644f0', upstream=None, plugins=None, cache=False, schema=Schema(identifier='schema', uuid='ea43dd0a76f749afb3b416925715c3c1', upstream=None, plugins=None, cache=False, fields={'x': FieldType(identifier='str', uuid='9426229dbd614d10b476e979a96f148f'), '_fold': FieldType(identifier='str', uuid='2de7977645d2435aacc4d8e6c1e1d573')}), primary_id='id'), Listener(identifier='chunker', uuid='chunk', upstream=None, plugins=None, cache=False, key='x', model=ObjectModel(identifier='chunker', uuid='3855aff339c84bf78af50c537c61b5ba', upstream=None, plugins=None, cache=False, signature='*args,**kwargs', datatype=None, output_schema=None, flatten=True, model_update_kwargs={'document_embedded': False}, predict_kwargs={}, compute_kwargs={}, validation=None, metric_values={}, num_workers=0, object=

In [16]:
application.info(verbosity=2)

2024-Aug-23 14:41:01.10| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf str already exists
2024-Aug-23 14:41:01.10| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf documents already exists
2024-Aug-23 14:41:01.10| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf schema already exists
2024-Aug-23 14:41:01.10| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf str already exists
2024-Aug-23 14:41:01.10| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf str already exists
2024-Aug-23 14:41:01.10| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf chunker already exists
2024-Aug-23 14:41:01.10| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf chunker already exists
2024-Aug-23 14:41:01.10| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf dill_lazy already exists
2024-Aug-23 14:41:01.10| 

╭────────────────────────────────────────────── text-vector-search ───────────────────────────────────────────────╮
│ identifier: text-vector-search                                                                                  │
│ uuid: ba67055ba77d40ffa73d5cd3a17118aa                                                                          │
│ cache: False                                                                                                    │
│ components: ["Table(identifier='documents', uuid='d05bd908810a43...", "Listener(identifier='chunker',           │
│ uuid='chunk', upstr...", "VectorIndex(identifier='my-vector-index', uuid='7c..."]                               │
│ namespace: ["{'type_id': 'table', 'identifier': 'documents'}", "{'type_id': 'listener', 'identifier':           │
│ 'chunker'}", "{'type_id': 'vector_index', 'identifier': 'my-vector-index'}", "{'type_id': 'schema',             │
│ 'identifier': 'schema'}", "{'type_id': 'model', 'identifier': 'chunker'}", "{'type_id': 'listener',             │
│ 'identifier': 'text-embedding-ada-002-listener'}", "{'type_id': 'model', 'identifier':                          │
│ 'text-embedding-ada-002'}"]                                                                                     │
│ _db: <superduper.base.datalayer.Datalayer object at 0x121816010>                                                │
│ version: 0                                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭────────────────────────────────────────────── Component Metadata ───────────────────────────────────────────────╮
│ Leaves                                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Component Map: text-vector-search
┣━━ str: superduper.components.schema.FieldType(2de7977645d2435aacc4d8e6c1e1d573)
┃   ┣━━ _path: superduper.components.schema.FieldType
┃   ┗━━ uuid: 2de7977645d2435aacc4d8e6c1e1d573
┣━━ schema: superduper.components.schema.Schema(ea43dd0a76f749afb3b416925715c3c1)
┃   ┣━━ _path: superduper.components.schema.Schema
┃   ┣━━ uuid: ea43dd0a76f749afb3b416925715c3c1
┃   ┣━━ upstream: None
┃   ┣━━ plugins: None
┃   ┣━━ cache: False
┃   ┣━━ fields: {'x': '?str', '_fold': '?str'}
┃   ┣━━ type_id: schema
┃   ┣━━ version: 0
┃   ┗━━ hidden: False
┣━━ documents: superduper.components.table.Table(d05bd908810a43988dbe41c2167644f0)
┃   ┣━━ _path: superduper.components.table.Table
┃   ┣━━ uuid: d05bd908810a43988dbe41c2167644f0
┃   ┣━━ upstream: None
┃   ┣━━ plugins: None
┃   ┣━━ cache: False
┃   ┣━━ schema: ?schema
┃   ┣━━ primary_id: id
┃   ┣━━ type_id: table
┃   ┣━━ version: 0
┃   ┗━━ hidden: False
┣━━ dill_lazy: superduper.components.datatype.get_serializer(0541e537287843c387e54b9f2ed666c5)
┃   ┣━━ _path: superduper.components.datatype.get_serializer
┃   ┣━━ method: dill
┃   ┣━━ encodable: lazy_artifact
┃   ┣━━ type_id: datatype
┃   ┣━━ version: None
┃   ┗━━ uuid: 0541e537287843c387e54b9f2ed666c5
┣━━ 03c2d3eed85dc1daff9b9ef78206e90a0f3e8f41: 
┃   superduper.components.datatype.LazyArtifact(07b8aead20534d35a2d1ae74bc02c97f)
┃   ┣━━ _path: superduper.components.datatype.LazyArtifact
┃   ┣━━ uuid: 07b8aead20534d35a2d1ae74bc02c97f
┃   ┣━━ datatype: ?dill_lazy
┃   ┣━━ uri: None
┃   ┗━━ blob: &:blob:03c2d3eed85dc1daff9b9ef78206e90a0f3e8f41
┣━━ chunker: superduper.components.listener.Listener(chunk)
┃   ┣━━ _path: superduper.components.listener.Listener
┃   ┣━━ uuid: chunk
┃   ┣━━ upstream: None
┃   ┣━━ plugins: None
┃   ┣━━ cache: False
┃   ┣━━ key: x
┃   ┣━━ model: ?chunker
┃   ┣━━ select: ?documents-select
┃   ┣━━ predict_kwargs: {}
┃   ┣━━ predict_id: chunker
┃   ┣━━ type_id: listener
┃   ┣━━ version: 0
┃   ┗━━ hidden: False
┣━━ documents-select: superduper_mongodb.query.parse_query(None)
┃   ┣━━ _path: superduper_mongodb.query.parse_query
┃   ┣━━ documents: []
┃   ┗━━ query: documents.select()
┣━━ vector[1536]: superduper.components.vector_index.vector(8189760e319d49caa494229916610895)
┃   ┣━━ _path: superduper.components.vector_index.vector
┃   ┣━━ shape: (1536,)
┃   ┣━━ type_id: datatype
┃   ┣━━ version: None
┃   ┗━━ uuid: 8189760e319d49caa494229916610895
┣━━ text-embedding-ada-002: superduper_openai.model.OpenAIEmbedding(31aa9703bbd740898bf6cd7112f41c69)
┃   ┣━━ _path: superduper_openai.model.OpenAIEmbedding
┃   ┣━━ uuid: 31aa9703bbd740898bf6cd7112f41c69
┃   ┣━━ upstream: None
┃   ┣━━ plugins: None
┃   ┣━━ cache: False
┃   ┣━━ signature: singleton
┃   ┣━━ datatype: ?vector[1536]
┃   ┣━━ output_schema: None
┃   ┣━━ flatten: False
┃   ┣━━ model_update_kwargs: {}
┃   ┣━━ predict_kwargs: {}
┃   ┣━━ compute_kwargs: {}
┃   ┣━━ validation: None
┃   ┣━━ metric_values: {}
┃   ┣━━ num_workers: 0
┃   ┣━━ model: text-embedding-ada-002
┃   ┣━━ max_batch_size: 8
┃   ┣━━ openai_api_key: None
┃   ┣━━ openai_api_base: None
┃   ┣━━ client_kwargs: {}
┃   ┣━━ shape: (1536,)
┃   ┣━━ batch_size: 100
┃   ┣━━ type_id: model
┃   ┣━━ version: 0
┃   ┗━━ hidden: False
┣━━ -outputs-chunker-select: superduper_mongodb.query.parse_query(None)
┃   ┣━━ _path: superduper_mongodb.query.parse_query
┃   ┣━━ documents: []
┃   ┗━━ query: _outputs__chunker.select()
┣━━ text-embedding-ada-002-listener: superduper.components.listener.Listener(ae8e2071d35342e585c5ca1f64067583)
┃   ┣━━ _path: superduper.components.listener.Listener
┃   ┣━━ uuid: ae8e2071d35342e585c5ca1f64067583
┃   ┣━━ upstream: ['?documents', '?chunker']
┃   ┣━━ plugins: None
┃   ┣━━ cache: False
┃   ┣━━ key: _outputs__chunker
┃   ┣━━ model: ?text-embedding-ada-002
┃   ┣━━ select: ?-outputs-chunker-select
┃   ┣━━ predict_kwargs: {}
┃   ┣━━ predict_id: text-embedding-ada-002-listener
┃   ┣━━ type_id: listener
┃   ┣━━ version: 0
┃   ┗━━ hidden: False
┣━━ my-vector-index: superduper.co

In [17]:
db['docs'].insert(datas).execute()
select = db['docs'].select()

2024-Aug-23 14:41:35.51| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:344  | Inserted 210 documents into documents
2024-Aug-23 14:41:35.52| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:404  | Created 210 events for insert on [documents]
2024-Aug-23 14:41:35.52| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:407  | Broadcasting 210 events
2024-Aug-23 14:41:35.52| INFO     | Duncans-MacBook-Pro.local| superduper.jobs.queue:210  | Running jobs for listener::chunker
2024-Aug-23 14:41:35.52| INFO     | Duncans-MacBook-Pro.local| superduper.backends.local.compute:67   | Submitting job. function:<function method_job at 0x112075300>
2024-Aug-23 14:41:35.52| INFO     | Duncans-MacBook-Pro.local| superduper.components.model:720  | Requesting prediction in db - [chunker] with predict_id chunker

2024-Aug-23 14:41:35.53| INFO     | Duncans-MacBook-Pro.local| superduper.components.model:853  | Adding 210 model outputs to `db`
2024-Aug-23 14:41:

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.67s/it]


2024-Aug-23 14:41:44.02| INFO     | Duncans-MacBook-Pro.local| superduper.components.model:853  | Adding 431 model outputs to `db`
2024-Aug-23 14:41:46.12| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:344  | Inserted 431 documents into _outputs__text-embedding-ada-002-listener
2024-Aug-23 14:41:46.15| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:404  | Created 431 events for insert on [_outputs__text-embedding-ada-002-listener]
2024-Aug-23 14:41:46.15| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:407  | Broadcasting 431 events
2024-Aug-23 14:41:46.15| INFO     | Duncans-MacBook-Pro.local| superduper.jobs.queue:210  | Running jobs for vector_index::my-vector-index
2024-Aug-23 14:41:46.15| INFO     | Duncans-MacBook-Pro.local| superduper.backends.local.compute:67   | Submitting job. function:<function callable_job at 0x1120753a0>
2024-Aug-23 14:41:46.54| SUCCESS  | Duncans-MacBook-Pro.local| superduper.backends.local.compute:73   |

In [19]:
db.databackend.db.list_collection_names()

['documents', '_outputs__chunker', '_outputs__text-embedding-ada-002-listener']

## Perform a vector search

In [21]:
from superduper import Document
# Perform the vector search based on the query
item = Document({indexing_key: "Tell me about vector-search"})

In [27]:
results = db[indexing_key].like(item, vector_index=vector_index_name, n=10).select().execute()

2024-Aug-23 14:43:39.94| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:889  | {}


In [28]:
for result in results:
    print("\n", '-' * 20, '\n')
    print(Document(result.unpack())[indexing_key])


 -------------------- 

--- sidebar_position: 7 --- # Vector-search SuperDuperDB allows users to implement vector-search in their database by either using in-database functionality, or via a sidecar implementation with `lance` and `FastAPI`. ## Philosophy In `superduperdb`, from a user point-of-view vector-search isn't a completely different beast than other ways of using the system: - The vector-preparation is exactly the same as preparing outputs with any model, with the special difference that the outputs are vectors, arrays or tensors. - Vector-searches are just another type of database query which happen to use the stored vectors. ## Algorithm Here is a schematic of how vector-search works: ![](/img/vector-search.png) ## Explanation A vector-search query has the schematic form: ```python table_or_collection .like(Document(<dict-to-search-with>)) # the operand is vectorized using registered models .filter_results(*args, **kwargs) # the results of vector-search are filtered ``` ```

In [33]:
from superduper import Template

t = Template(
    'vector-search',
    template=application,
    substitutions={'docs': 'table_name'},
)

2024-Aug-23 14:47:05.73| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf str already exists
2024-Aug-23 14:47:05.74| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf documents already exists
2024-Aug-23 14:47:05.74| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf schema already exists
2024-Aug-23 14:47:05.74| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf str already exists
2024-Aug-23 14:47:05.74| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf str already exists
2024-Aug-23 14:47:05.74| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf chunker already exists
2024-Aug-23 14:47:05.74| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf chunker already exists
2024-Aug-23 14:47:05.74| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:415  | Leaf dill_lazy already exists
2024-Aug-23 14:47:05.74| 

In [35]:
t.export('.')

In [37]:
!cat component.json | jq .

{
  "_base": "?vector-search",
  "_builds": {
    "vector-search": {
      "_path": "superduper.components.template.Template",
      "template": {
        "_base": "?text-vector-search",
        "_builds": {
          "str": {
            "_path": "superduper.components.schema.FieldType"
          },
          "schema": {
            "_path": "superduper.components.schema.Schema",
            "fields": {
              "x": "?str",
              "_fold": "?str"
            }
          },
          "<var:table_name>": {
            "_path": "superduper.components.table.Table",
            "schema": "?schema"
          },
          "dill_lazy": {
            "_path": "superduper.components.datatype.get_serializer",
            "method": "dill",
            "encodable": "lazy_artifact"
          },
          "03c2d3eed85dc1daff9b9ef78206e90a0f3e8f41": {
            "_path": "superduper.components.datatype.LazyArtifact",
            "datatype": "?dill_lazy",
            "blob": "&:blob:03c2